In [20]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [21]:
model=load_model('model.h5')
with open('label_encoder_gender.pkl','rb') as file:
    label_enocder_gender=pickle.load(file)
with open('scaler.pkl','rb') as file:
    scaler=pickle.load(file)
with open('ohe.pkl','rb') as file:
    label_enocder_geo=pickle.load(file)


In [22]:
input_data={
    'CreditScore':600,
    'Geography':'France',
    'Gender':'Male',
    'Age':23,
    'Tenure':3,
    'Balance':6000,
    'NumOfProducts':2,
    'HasCrCard':1,
    'IsActiveMember':0,
    'EstimatedSalary':5000
}

In [23]:
geo_encoded=label_enocder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df=pd.DataFrame(geo_encoded,columns=label_enocder_geo.get_feature_names_out(['Geography']))

c:\Program Files\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [24]:
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [25]:
input_df=pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,23,3,6000,2,1,0,5000


In [26]:
input_df['Gender']=label_enocder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,23,3,6000,2,1,0,5000


In [27]:
input_df=pd.concat([input_df.drop(columns=['Geography'],axis=1),geo_encoded_df],axis=1)

In [28]:
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,23,3,6000,2,1,0,5000,1.0,0.0,0.0


In [29]:
input_scaled=scaler.transform(input_df)
input_scaled

array([[-0.53598516,  0.91324755, -1.51143782, -0.69539349, -1.12240462,
         0.80843615,  0.64920267, -1.02583358, -1.65923237,  1.00150113,
        -0.57946723, -0.57638802]])

In [30]:
prediction=model.predict(input_scaled)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step


In [31]:
prediction

array([[0.00331265]], dtype=float32)

In [32]:
prediction_prob=prediction[0][0]
prediction_prob

np.float32(0.0033126522)

In [33]:
if prediction_prob>0.5:
    print('The Customer is Likely to Churn')
else:
    print('The Customer is not Likely to Churn')

The Customer is not Likely to Churn
